In [106]:
import os
import pandas as pd

In [ ]:
# 파일명 변경

folder_names = os.listdir('./광주/')

for folder_name in folder_names:
    file_names = os.listdir(f'./광주/{folder_name}')
    for file_name in file_names:
        split_file_name = file_name.replace('.csv','').split('_')
        os.rename(f'./광주/{folder_name}/{file_name}',f'./광주/{folder_name}/{folder_name}_{split_file_name[1]}.csv')

### 데이터프레임 합치기
### 관광지 기본정보

In [ ]:
basic_info = pd.DataFrame()

for folder_name in folder_names:
    df = pd.read_csv(f'./광주/{folder_name}/{folder_name}_관광지 기본정보.csv',encoding='ANSI')
    basic_info = pd.concat([basic_info,df],ignore_index=True)

In [ ]:
basic_info

In [ ]:
basic_info.to_csv('./광주/basic_info.csv',encoding='utf-8',index=False)

### 데이터프레임 합치기

### 내비게이션

In [ ]:
navi_info = pd.DataFrame()

for folder_name in folder_names:
    df = pd.read_csv(f'./광주/{folder_name}/{folder_name}_내비게이션 목적지 검색량 추이.csv',encoding='ANSI',index_col=0)
    df = df.rename(columns={df.columns[0] : folder_name}).transpose()
    df['관광지명'] = folder_name
    navi_info = pd.concat([navi_info,df])

In [ ]:
order = [202401,202302, 202303, 202304,202305, 202306, 202307, 202308,202309, 202310, 202311, 202312, '관광지명']
navi_info = navi_info[order]

In [134]:
order = [202301,202302, 202303, 202304,202305, 202306, 202307, 202308,202309, 202310, 202311, 202312, '관광지명']
navi_info.columns = order

In [136]:
order = ['관광지명',202301,202302, 202303, 202304,202305, 202306, 202307, 202308,202309, 202310, 202311, 202312]
navi_info = navi_info[order]

In [138]:
navi_info.to_csv('./광주/navi_info.csv',encoding='utf-8',index=False)

### 데이터프레임 합치기
### 방문자 거주지별 분포

In [ ]:
order = ['관광지명','광역지자체명', '기초지자체명', '광역지자체별 거주 방문자 비율', '기초지자체별 거주 방문자 비율']

visit_info = pd.DataFrame()

for folder_name in folder_names:
    df = pd.read_csv(f'./광주/{folder_name}/{folder_name}_방문자 거주지별 분포.csv',encoding='ANSI')
    df['관광지명'] = folder_name
    df = df[order]
    
    visit_info = pd.concat([visit_info,df],ignore_index=True)

In [ ]:
visit_info.to_csv('./광주/visit_info.csv',encoding='utf-8',index=False)

### 데이터프레임 합치기
### 성별

In [ ]:
order = ['관광지명','연령대', '남성방문 비율', '여성방문 비율']

gender_info = pd.DataFrame()

for folder_name in folder_names:
    df = pd.read_csv(f'./광주/{folder_name}/{folder_name}_성·연령별 방문자.csv',encoding='ANSI')
    df['관광지명'] = folder_name
    df = df[order]
    gender_info = pd.concat([gender_info,df],ignore_index=True)

In [ ]:
gender_info

,관광지명,연령대,남성방문 비율,여성방문 비율
0,1913송정역시장,70세 이상,4.3,3.0
1,1913송정역시장,60~69세,8.8,5.4
2,1913송정역시장,50~59세,12.5,8.5
3,1913송정역시장,40~49세,9.8,5.9
4,1913송정역시장,30~39세,9.9,7.2
...,...,...,...,...
387,환벽당,40~49세,7.6,7.0
388,환벽당,30~39세,4.2,4.9
389,환벽당,20~29세,4.7,5.2
390,환벽당,10~19세,1.2,1.1


In [ ]:
gender_info.to_csv('./광주/gender_info.csv',encoding='utf-8',index=False)

### 데이터 가져오기
### 위치정보

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
url = 'https://datalab.visitkorea.or.kr/visualize/getTempleteData.do'

params = {
    "SIDO_CD": 29,
    "SGG_CD": "",
    "BASE_YM1": 2010,
    "BASE_YM2": 2022,
    "CONT_NM": "",
    "KTO_CATE_LCLS_CD": "",
    "KTO_CATE_MCLS_CD": "",
    "firstIndex": float('nan'),
    "pageIndex": 1,
    "recordCountPerPage": 10,
    "orderSelect": "",
    "orderValue": "asc",
    "contentList": "전체",
    "noTheme": "",
    "qid": "LN_05_01_011"
}

response = requests.get(url,params=params)
data = response.json()

In [ ]:
all = []

while True:
    for item in data.get('list', []):
        result = {
            '관광지명': item.get('CONT_NM'),
            '주소': item.get('TAR_ADDR_NM'),
            '대분류': item.get('KTO_CATE_LCLS_NM'),
            '중분류': item.get('KTO_CATE_MCLS_NM'),
            '소분류': item.get('KTO_CATE_SCLS_NM'),
            'longitude': item.get('XCRD'),
            'latitude': item.get('YCRD')
        }
        all.append(result)
        
    params['pageIndex'] += 1
    
    time.sleep(5)
    
    if len(all) == 49:
        break

In [ ]:
df = pd.DataFrame(all)

In [ ]:
df.to_csv('./광주/location_info.csv',encoding='utf-8',index=False)

==================================================================================================

==================================================================================================

==================================================================================================

In [ ]:
import pandas as pd

basic_info = pd.read_csv('./광주/basic_info.csv')
gender_info = pd.read_csv('./광주/gender_info.csv')
location_info = pd.read_csv('./광주/location_info.csv')
navi_info = pd.read_csv('./광주/navi_info.csv')
visit_info = pd.read_csv('./광주/visit_info.csv')

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

stars = []
reviews = []

for i in basic_info['관광지명']:
    url = f'https://www.google.co.kr/maps/search/{i}'
    service = Service(executable_path='./data/chromedriver.exe')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    
    try:
        # 해당 클래스명이 나타날 때까지 최대 10초 대기
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'F7nice'))
        )
        
        star = element.text.split('\n')[0]
        review = element.text.split('\n')[1][1:-1]
        
        stars.append(star)
        reviews.append(review)
    except:
        check = input('일치하는 장소를 클릭 후 y를 눌러주세요(이외 None)')
        if check.upper() == 'Y':
            star = driver.find_element(By.CLASS_NAME, 'F7nice').text.split('\n')[0]
            review = driver.find_element(By.CLASS_NAME, 'F7nice').text.split('\n')[1][1:-1]
            
            stars.append(star)
            reviews.append(review)
        else:
            stars.append(None)
            reviews.append(None)
    
    driver.quit()
    print(f'평점 수 : {len(stars)}','|',f'리뷰 수 : {len(reviews)}','|',f'관광지 명 : {i}','|', f'별점 : {star}','|',f'리뷰 : {review}')
    
    star = None
    review = None

평점 수 : 1 | 리뷰 수 : 1 | 관광지 명 : 1913송정역시장 | 별점 : 3.9 | 리뷰 : 4,489
평점 수 : 2 | 리뷰 수 : 2 | 관광지 명 : 5·18기념공원 | 별점 : 4.4 | 리뷰 : 654
평점 수 : 3 | 리뷰 수 : 3 | 관광지 명 : 5·18민주화운동기록관 | 별점 : 4.1 | 리뷰 : 9
평점 수 : 4 | 리뷰 수 : 4 | 관광지 명 : 고싸움놀이전수회관 | 별점 : 4.1 | 리뷰 : 7
평점 수 : 5 | 리뷰 수 : 5 | 관광지 명 : 광주공원 | 별점 : 4.1 | 리뷰 : 445
평점 수 : 6 | 리뷰 수 : 6 | 관광지 명 : 광주대인시장(대인예술시장) | 별점 : 3.9 | 리뷰 : 1,288
평점 수 : 7 | 리뷰 수 : 7 | 관광지 명 : 광주비엔날레전시관 | 별점 : 4.3 | 리뷰 : 261
평점 수 : 8 | 리뷰 수 : 8 | 관광지 명 : 광주사직공원 | 별점 : 4.3 | 리뷰 : 985
평점 수 : 9 | 리뷰 수 : 9 | 관광지 명 : 광주서강사 | 별점 : None | 리뷰 : None
평점 수 : 10 | 리뷰 수 : 10 | 관광지 명 : 광주송정공원 | 별점 : 4.5 | 리뷰 : 22
평점 수 : 11 | 리뷰 수 : 11 | 관광지 명 : 광주수완호수공원 | 별점 : 4.1 | 리뷰 : 560
평점 수 : 12 | 리뷰 수 : 12 | 관광지 명 : 광주시립미술관 | 별점 : 4.4 | 리뷰 : 1,031
평점 수 : 13 | 리뷰 수 : 13 | 관광지 명 : 광주시민의숲 | 별점 : 3.9 | 리뷰 : 7
평점 수 : 14 | 리뷰 수 : 14 | 관광지 명 : 광주양동시장 | 별점 : 4.1 | 리뷰 : 900
평점 수 : 15 | 리뷰 수 : 15 | 관광지 명 : 광주역사민속박물관 | 별점 : 4.3 | 리뷰 : 407
평점 수 : 16 | 리뷰 수 : 16 | 관광지 명 : 광주예술의거리 | 별점 : 4.1 | 리뷰 : 153
평점 수 : 17 | 

In [ ]:
print(f'총 데이터 수 : {len(basic_info)}')
print('=========================================')
print(f'평점 수 : {len(stars)}')
print(f'리뷰 수 : {len(reviews)}')
print('=========================================')
print(f'총 None값 수 : {stars.count(None)}')
print(f'총 None값 수 : {reviews.count(None)}')

총 데이터 수 : 49
평점 수 : 49
리뷰 수 : 49
총 None값 수 : 2
총 None값 수 : 2


In [ ]:
basic_info['stars'] = stars
basic_info['reviews'] = reviews

In [ ]:
basic_info.to_csv('./광주/basic_info.csv',encoding='utf-8',index=False)

### 리뷰, 체류시간 -> 숫자 값으로 변경

In [ ]:
time = []

for i in basic_info['평균 체류시간']:
    i = i.replace('시간','').replace('분','').strip()
    i = i.split(' ')
    time.append((int(i[0]) * 60) + int(i[1]))
    
basic_info['평균 체류시간'] = time

In [ ]:
reviews_to_int = []

for i in basic_info['reviews']:
    if type(i) == type(None):
        reviews_to_int.append(i)
        continue
    if type(i) in (float,int):
        reviews_to_int.append(i)
    else:
        i = i.replace(',','')
        i = int(i)
        reviews_to_int.append(i)

basic_info['reviews'] = reviews_to_int

In [ ]:
basic_info.to_csv('./광주/basic_info.csv',encoding='utf-8',index=False)

### None값 -> 관광지별 평균

In [ ]:
navi_info.set_index('관광지명',inplace=True)

In [ ]:
navi_info = navi_info.apply(lambda row: row.fillna(int(row.mean())), axis=1)

In [ ]:
navi_info.to_csv('./광주/navi_info.csv',encoding='utf-8',index=False)

### 광역지자체별 거주 방문자 비율

In [ ]:
visit_info.dropna(subset=['광역지자체명'], axis=0,inplace=True)

In [ ]:
names = visit_info['관광지명'].tolist()

local_visit = visit_info[visit_info['광역지자체명'].str.contains('광주')].groupby('관광지명')[['광역지자체별 거주 방문자 비율']].mean()
local_visit = local_visit.reset_index()

local_visit.loc[local_visit['관광지명'].isin(names)]

In [ ]:
basic_info['광역지자체별 거주 방문자 비율'] = local_visit['광역지자체별 거주 방문자 비율']

In [ ]:
basic_info

===============================================================================================================

===============================================================================================================

===============================================================================================================

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

search_number = []
for i in basic_info['관광지명']:
    if not i.startswith('광주'):
        i = '광주 ' + i
    url = f'https://www.google.com/search?q={i}'
    service = Service(executable_path='./data/chromedriver.exe')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    time.sleep(2)

    search_number.append(driver.find_element(By.ID, 'result-stats').text)

basic_info['search_number'] = search_number

In [ ]:
search_number = []
for index,i in enumerate(basic_info['관광지명']):
    if basic_info.iloc[index]['search_number']:
        continue
    if not i.startswith('광주'):
        i = '광주 ' + i
    url = f'https://www.google.com/search?q={i}'
    service = Service(executable_path='./data/chromedriver.exe')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    time.sleep(2)

    # search_number.append(driver.find_element(By.ID, 'result-stats').text)
    basic_info.loc[index,'search_number'] = driver.find_element(By.ID, 'result-stats').text

In [ ]:
import re

pattern = re.compile(r'\([^)]*\)')

basic_info['search_number'] = basic_info['search_number'].apply(lambda x: re.sub(pattern,'',x).strip())

basic_info['search_number'] = basic_info['search_number'].apply(lambda x: int(x.replace('개','').replace(',','')[7:]))

In [ ]:
basic_info.to_csv('./광주/basic_info.csv',encoding='utf-8',index=False)

===============================================================================================================

===============================================================================================================

===============================================================================================================

In [1]:
import pandas as pd

basic_info = pd.read_csv('./광주/basic_info.csv')
gender_info = pd.read_csv('./광주/gender_info.csv')
location_info = pd.read_csv('./광주/location_info.csv')
navi_info = pd.read_csv('./광주/navi_info.csv')
visit_info = pd.read_csv('./광주/visit_info.csv')

In [2]:
totals = []
avgs = []

for i in range(len(navi_info)):
    total = navi_info.iloc[i][1:].sum()
    avg = navi_info.iloc[i][1:].mean()
    
    totals.append(total)
    avgs.append(avg)
navi_info['sum'] = totals
navi_info['avg'] = avgs

In [3]:
df = pd.DataFrame()
df = pd.concat([basic_info,navi_info[['sum','avg']]],axis=1)
df = pd.merge(df,location_info,on='관광지명',how='inner')

In [4]:
import numpy as np
score = []

for i in range(len(df)):
    if pd.isna(df.iloc[i]['stars']):
        star = 0
    else:
        star = df.iloc[i]['stars'] * 1.2
        
    if df.iloc[i]['reviews']:
        review = 0
    else:
        review = np.log1p(max(df['reviews'])) - np.log1p(df.iloc[i]['reviews']) * 1.1
    search = np.log1p(max(df['search_number'])) - np.log1p(df.iloc[i]['search_number'])
    if pd.isna(df.iloc[0]['광역지자체별 거주 방문자 비율']):
        visit = 0
    else:
        visit = np.log1p(df.iloc[i]['광역지자체별 거주 방문자 비율']) * 1.1
    time = np.log1p(df.iloc[i]['평균 체류시간']) * 0.8
    navi = np.log1p(max(df['avg'])) - np.log1p(df.iloc[i]['avg'])
    all = star+review+search+visit+time-navi
    
    score.append(all)

df['scores'] = score

In [5]:
first = df.sort_values(by='scores',ascending=False)

In [7]:
first.to_csv('./gwangju.csv',encoding='utf-8',index=False)

In [ ]:
first['관광지명'].tail(25).tolist()

In [ ]:
first['관광지명'].head(25).tolist()

In [166]:
import folium

gwangju_map = folium.Map(location=[35.1599785,126.8513072],zoom_start=11)

In [167]:
for i in range(len(first)):
    if i == 30:
        break
    folium.Marker(
            [first.iloc[i]['latitude'],first.iloc[i]['longitude']],
            popup=folium.Popup(first.iloc[i]['관광지명'],maxWidth=300),
            fill_opacity=0.5,
            icon=folium.Icon(color='blue')            
            ).add_to(gwangju_map)

In [ ]:
gwangju_map

In [118]:
import pandas as pd
import os

In [119]:
# 파일이 있는 디렉토리 설정
directory = "./광주/대중교통"

# 모든 CSV 파일 목록 가져오기
file_list = [file for file in os.listdir(directory) if file.endswith('.csv')]

# 모든 CSV 파일을 담을 빈 DataFrame 생성
combined_df = pd.DataFrame()

# 각 CSV 파일을 순회하면서 데이터를 합치기
for file in file_list:
    file_path = os.path.join(directory, file)
    
    # 관광지 명 추출
    landmark_name = file.split('_')[1]  # 파일명에서 '_'를 기준으로 분리하여 첫 번째 요소를 관광지 명으로 사용
    landmark_name = landmark_name.replace('.csv', '')
    
    # CSV 파일을 DataFrame으로 읽기
    df = pd.read_csv(file_path, encoding="cp949")
    
    # 관광지 명 컬럼 추가
    df['관광지명'] = landmark_name
    
    # 현재 파일의 데이터를 합치기
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# 결과를 CSV 파일로 저장하기
# combined_df.to_csv('combined_data.csv', index=False, encoding="utf-8")

# print("합쳐진 데이터를 combined_data.csv로 저장했습니다.")

In [120]:
total_column = []
for i in range(len(combined_df)):
    total = combined_df.iloc[i][6:-2].sum()
    total_column.append(total)

combined_df['total'] = total_column

In [121]:
# 관광지명으로 그룹화하여 총 승객수를 구하고 새로운 컬럼으로 추가
total_passengers_by_landmark = combined_df.groupby('관광지명')['total'].sum().reset_index()
total_passengers_by_landmark.rename(columns={'total': '총 승객수'}, inplace=True)

# 기존 데이터프레임에 총 승객수 컬럼 추가
df = pd.merge(combined_df, total_passengers_by_landmark, on='관광지명', how='left')

In [122]:
bus = df.groupby('관광지명')[['노선']].nunique().reset_index()

In [123]:
new_df = df[['관광지명', '총 승객수']]
new_df = new_df.drop_duplicates()

In [124]:
new_df = pd.merge(new_df,bus,on='관광지명',how='inner')

In [125]:
for i in range(len(new_df)):
    if new_df.iloc[i]['관광지명'] in first.head(25)['관광지명'].tolist():
        new_df.loc[i,'구분'] = '숨은 명소'
    else:
        new_df.loc[i,'구분'] = '일반 명소'

In [137]:
len(basic_info)

49

In [126]:
print(len(new_df[new_df['구분'] == '일반 명소'].sort_values(by='총 승객수',ascending=False)))
print(len(new_df[new_df['구분'] == '숨은 명소'].sort_values(by='총 승객수',ascending=False)))

15
15


In [134]:
new_df[new_df['구분'] == '일반 명소']

,관광지명,총 승객수,노선,구분
1,5·18기념공원,2266,3,일반 명소
4,국립광주박물관,688,3,일반 명소
5,광주시민의숲,257,1,일반 명소
6,오리요리의거리,55356,11,일반 명소
7,광주사직공원,15998,5,일반 명소
9,신광사(광주),5658,5,일반 명소
10,동명동카페거리,2893,4,일반 명소
13,광주서광사,12995,8,일반 명소
14,푸른길분수공원,12995,8,일반 명소
16,약사암(광주),576,2,일반 명소


In [135]:
new_df[new_df['구분'] == '숨은 명소']

,관광지명,총 승객수,노선,구분
0,김대중컨벤션센터,4473,6,숨은 명소
2,광주호호수생태원,133,2,숨은 명소
3,국립광주과학관,906,8,숨은 명소
8,광주대인시장(대인예술시장),8833,11,숨은 명소
11,송산유원지,1430,3,숨은 명소
12,증심사(광주),136,1,숨은 명소
15,광주비엔날레전시관,3658,3,숨은 명소
19,빛고을농촌테마공원,24,3,숨은 명소
22,광주패밀리랜드,1507,5,숨은 명소
23,무각사,13626,5,숨은 명소


In [129]:
from scipy import stats

test_names = ['총 승객수','노선']

p = new_df[new_df['구분'] == '일반 명소'][['총 승객수','노선']]
up = new_df[new_df['구분'] == '숨은 명소'][['총 승객수','노선']]

for test_name in test_names:
    statistic,p_value = stats.levene(p[test_name], up[test_name])
    if p_value > 0.05:
        print(f"{test_name} p-value: {p_value}, 등분산 가정 만족")
    else:
        print(f"{test_name} p-value: {p_value}, 이분산 가정 만족")

총 승객수 p-value: 0.028710059036753034, 이분산 가정 만족
노선 p-value: 0.3037958515807606, 등분산 가정 만족


In [130]:
for test_name in test_names[0:1]:
    t_statistic,pvalue = stats.ttest_ind(
        a = p[test_name],
        b = up[test_name],
        alternative = 'two-sided',
        equal_var = False
)
    if pvalue<0.05:
        print(test_name,pvalue,'귀무가설 기각')
    else:
        print(test_name,pvalue,'귀무가설 기각 불가')

총 승객수 0.03848493616035898 귀무가설 기각


In [131]:
for test_name in test_names[1:]:
    t_statistic,pvalue = stats.ttest_ind(
        a = p[test_name],
        b = up[test_name],
        alternative = 'two-sided',
        equal_var = True
)
    if pvalue<0.05:
        print(test_name,pvalue,'귀무가설 기각')
    else:
        print(test_name,pvalue,'귀무가설 기각 불가')

노선 0.16535680219011004 귀무가설 기각 불가


In [132]:
new_df.to_csv('./광주/bus_info.csv',encoding='utf-8',index=False)

======================================================================================================================================================

======================================================================================================================================================

======================================================================================================================================================

In [9]:
import requests
from bs4 import BeautifulSoup

In [10]:
url = 'https://korean.visitkorea.or.kr/call'
params = {
    'cmd': 'TOUR_CONTENT_LIST_VIEW',
    'month': 'All',
    'areaCode': '5',
    'sigunguCode': 'All',
    'tagId': 'All',
    'sortkind': '1',
    'locationx': '0',
    'locationy': '0',
    'page': '1',
    'cnt': '100',
    'typeList': 'Tour',
    'stampId': '1589345b-b030-11ea-b8bd-020027310001'
}

all_data = []  # 모든 페이지의 데이터를 저장할 리스트

while True:
    response = requests.get(url, params=params)
    data = response.json()

    # 받은 데이터에서 필요한 정보 추출
    result_list = data.get('body', {}).get('result', [])
    for item in result_list:
        img_path = item.get('imgPath', '')
        img_url = f'https://cdn.visitkorea.or.kr/img/call?cmd=VIEW&id={img_path}'
        
        extracted_data = {
            '관광지명': item.get('TITLE', ''),
            # 'catch': item.get('catchPhrase', ''),
            # 'addr1': item.get('addr1', ''),
            'img': img_url,
            # 'cotid' : item.get('cotId', ''),
            # 'areaCode' : item.get('areaCode', ''),
            # 'cat1' : item.get('cat1', ''),
            # 'cat2' : item.get('cat2', '')
        }
        
        # cotid = extracted_data['cotid']
        # cat1 = extracted_data['cat1']
        # cat2 = extracted_data['cat2']
        # areaCode = extracted_data['areaCode']
        
        # new_url = f'https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid={cotid}&big_category={cat1}&mid_category={cat2}&big_area={areaCode}'
        # response = requests.get(new_url)
        # html = response.text
        # soup = BeautifulSoup(html,'html.parser')
        # description = soup.find('div',class_='blind').text
        # description = description.replace('\n','').replace('\t','')
        # extracted_data['description'] = description

        all_data.append(extracted_data)
    
    # 다음 페이지로 이동
    # if params['page'] in range(0,1000,100):
    #     print('time sleep')
    #     time.sleep(200)
    # else:
    
    print(params['page'])
    params['page'] = str(int(params['page']) + 1)
    

    # 만약 더 이상 페이지가 없으면 반복문 종료
    if not result_list or len(all_data) == 493:
        break

1
2
3
4
5


In [3]:
import pandas as pd

g = pd.read_csv('./data/gwangju.csv')
gg = pd.read_csv('./광주/basic_info.csv')

In [7]:
g = pd.merge(g,gg[['관광지명','catch','description']],on='관광지명',how='left')

In [ ]:
df = pd.DataFrame(all_data)

In [13]:
import re
from tqdm import tqdmly(lambda x: re.sub(pattern, '', x))

tqdm.pandas()
df['관광지명'] = df['관광지명'].progress_apply(lambda x: x.replace(' ',''))

pattern = re.compile(r'\([^)]*\)')
df['관광지명'] = df['관광지명'].app

100%|██████████| 493/493 [00:00<00:00, 492448.65it/s]


In [16]:
g = pd.merge(g,df,on='관광지명',how='left')

In [36]:
g[g.img.isna()][['관광지명']]

,관광지명


In [35]:
g.loc[18,'img'] = df.iloc[41]['img']
g.loc[17,'img'] = df.iloc[85]['img']
g.loc[30,'img'] = df.iloc[149]['img']
g.loc[48,'img'] = df.iloc[38]['img']
g.loc[42,'img'] = df.iloc[351]['img']
g.loc[1,'img'] = df.iloc[40]['img']
g.loc[47,'img'] = df.iloc[11]['img']
g.loc[26,'img'] = df.iloc[27]['img']
g.loc[40,'img'] = df.iloc[449]['img']
g.loc[14,'img'] = df.iloc[83]['img']
g.loc[19,'img'] = df.iloc[356]['img']
g.loc[25,'img'] = df.iloc[138]['img']

In [42]:
g = g.drop(['주소_x','sum'],axis=1).rename(columns={'주소_y':'주소'})

In [43]:
g.to_csv('./data/gwangju.csv',encoding='utf-8',index=False)

In [1]:
import pandas as pd

df = pd.read_csv('./광주/bus_info.csv')

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import re

In [10]:
for i in range(len(df)):
    url = 'https://map.naver.com/p/directions/14127476.0146588,4186379.4066277,%EA%B4%91%EC%A3%BC%EC%97%AD%20%EA%B4%91%EC%A3%BC%EC%84%A0,11630524,PLACE_POI/-/-/transit?c=14.20,0,0,0,dh'
    service = Service(executable_path='./data/chromedriver.exe')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    time.sleep(3)
    
    try : 
        for j in range(0, 5):
            elements = driver.find_elements(By.ID, f'input_search:r{j}:')
            if not elements:
                continue
            if elements:
                if df.iloc[i]['관광지명'].startswith('광주'):
                    time.sleep(3)
                    driver.find_element(By.ID,f'input_search:r{j+1}:').send_keys(df.iloc[i]['관광지명'])
                else:
                    time.sleep(3)
                    driver.find_element(By.ID,f'input_search:r{j+1}:').send_keys('광주',' ', df.iloc[i]['관광지명'])

                time.sleep(3)
                driver.find_element(By.ID,f'input_search:r{j+1}:').send_keys(Keys.ENTER)
                time.sleep(2)
                place_click = driver.find_element(By.CSS_SELECTOR,"button.btn_direction.search.active").click()
                time.sleep(2)
                
                a = driver.find_element(By.CLASS_NAME,'item_btn').text
                
                total_time = re.search(r"최적\n(\d+시간\d+분|\d+분|\d+시간)", a).group(1)
                walking_time = sum(map(int,re.findall(r'도보\n(\d+)', a)))
                if re.search(r'([\d,]+)원', a):
                    fare = re.search(r'([\d,]+)원', a).group(1)
                else:
                    fare = None
                bus_time = sum(int(i) for i in re.findall(r'버스\n(\d+)', a[:a.find('도착정보')]))
                subway_time = sum(int(l) for l in re.findall(r'지하철\n(\d+?)분', a[:a.find('도착정보')]))
                transfer = len(re.findall(r'버스\n(\d+)', a[:a.find('도착정보')])) + len(re.findall(r'지하철\n(\d+?)분', a[:a.find('도착정보')])) -1
                
                df.at[i,'total_time'] = total_time
                df.at[i,'walking_time'] = walking_time
                df.at[i,'fare'] = fare
                df.at[i,'bus_time'] = bus_time
                df.at[i,'subway_time'] = subway_time
                df.at[i,'transfer'] = transfer
            break
    except : 
        check = input('장소 클릭')
        if check.upper() == 'Y':
            a = driver.find_element(By.CLASS_NAME,'item_btn').text
            
            total_time = re.search(r"최적\n(\d+시간\d+분|\d+분|\d+시간)", a).group(1)
            walking_time = sum(map(int,re.findall(r'도보\n(\d+)', a)))
            if re.search(r'([\d,]+)원', a):
                fare = re.search(r'([\d,]+)원', a).group(1)
            else:
                fare = None
            bus_time = sum(int(i) for i in re.findall(r'버스\n(\d+)', a[:a.find('도착정보')]))
            subway_time = sum(int(l) for l in re.findall(r'지하철\n(\d+?)분', a[:a.find('도착정보')]))
            transfer = len(re.findall(r'버스\n(\d+)', a[:a.find('도착정보')])) + len(re.findall(r'지하철\n(\d+?)분', a[:a.find('도착정보')])) -1
                
            df.at[i,'total_time'] = total_time
            df.at[i,'walking_time'] = walking_time
            df.at[i,'fare'] = fare
            df.at[i,'bus_time'] = bus_time
            df.at[i,'subway_time'] = subway_time
            df.at[i,'transfer'] = transfer
        else:
            df.at[i,'total_time'] = None
            df.at[i,'walking_time'] = None
            df.at[i,'fare'] = None
            df.at[i,'bus_time'] = None
            df.at[i,'subway_time'] = None
            df.at[i,'transfer'] = None

    print(f"{df.iloc[i]['관광지명']} :: 총:{total_time} 버스:{bus_time} 지하철:{subway_time} 걷가:{walking_time} 요금:{fare} 환승:{transfer}")
    driver.quit()

김대중컨벤션센터 :: 총:33분 버스:24 지하철:0 걷가:9 요금:1,250 환승:0
5·18기념공원 :: 총:29분 버스:18 지하철:0 걷가:12 요금:1,700 환승:1
광주호호수생태원 :: 총:49분 버스:33 지하철:0 걷가:6 요금:None 환승:1
국립광주과학관 :: 총:44분 버스:32 지하철:0 걷가:11 요금:None 환승:1
국립광주박물관 :: 총:19분 버스:12 지하철:0 걷가:5 요금:1,250 환승:1
광주시민의숲 :: 총:54분 버스:33 지하철:0 걷가:20 요금:1,250 환승:1
오리요리의거리 :: 총:15분 버스:3 지하철:0 걷가:11 요금:1,250 환승:0
광주사직공원 :: 총:31분 버스:15 지하철:0 걷가:15 요금:1,250 환승:1
광주대인시장(대인예술시장) :: 총:11분 버스:6 지하철:0 걷가:6 요금:1,250 환승:0
신광사(광주) :: 총:32분 버스:8 지하철:0 걷가:24 요금:1,250 환승:0
동명동카페거리 :: 총:18분 버스:6 지하철:0 걷가:11 요금:1,250 환승:0
송산유원지 :: 총:57분 버스:46 지하철:0 걷가:10 요금:1,250 환승:1
증심사(광주) :: 총:21분 버스:14 지하철:0 걷가:6 요금:1,250 환승:0
광주서광사 :: 총:23분 버스:12 지하철:0 걷가:11 요금:1,250 환승:0
푸른길분수공원 :: 총:19분 버스:12 지하철:0 걷가:8 요금:1,250 환승:0
광주비엔날레전시관 :: 총:26분 버스:12 지하철:0 걷가:13 요금:1,250 환승:1
약사암(광주) :: 총:1시간27분 버스:23 지하철:0 걷가:64 요금:1,700 환승:0
광주송정공원 :: 총:47분 버스:39 지하철:0 걷가:8 요금:1,250 환승:1
광주수완호수공원 :: 총:41분 버스:29 지하철:0 걷가:12 요금:1,250 환승:0
빛고을농촌테마공원 :: 총:52분 버스:38 지하철:0 걷가:13 요금:1,250 환승:1
광주시립미술관 :: 총:27분 버스:9 

In [6]:
url = 'https://map.naver.com/p/directions/14127476.0146588,4186379.4066277,%EA%B4%91%EC%A3%BC%EC%97%AD%20%EA%B4%91%EC%A3%BC%EC%84%A0,11630524,PLACE_POI/-/-/transit?c=14.20,0,0,0,dh'
service = Service(executable_path='./data/chromedriver.exe')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(url)
time.sleep(3)

In [12]:
df.to_csv('./광주/bus_info.csv',encoding='utf-8',index=False)